In [222]:
import pandas as pd
import numpy as np
import requests
import json
import os


from bs4 import BeautifulSoup
from rapidfuzz import process, fuzz  # Fuzzy matching for names
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from rapidfuzz import process, fuzz

In [223]:
team_names = {
            'Arsenal': 'Arsenal', 'Aston Villa': 'Aston Villa', 'Bournemouth': 'Bournemouth', 'Brentford': 'Brentford', 'Brighton': 'Brighton',
            'Chelsea': 'Chelsea', 'Crystal Palace': 'Crystal Palace', 'Everton': 'Everton', 'Fulham': 'Fulham', 'Ipswich': 'Ipswich',
            'Leicester': 'Leicester', 'Liverpool': 'Liverpool', 'Man City': 'Manchester City', 'Man Utd': 'Manchester United', 'Newcastle': 'Newcastle United',
            "Nott'm Forest": 'Nottingham Forest', 'Southampton': 'Southampton', 'Spurs': 'Tottenham', 'West Ham': 'West Ham', 'Wolves': 'Wolverhampton Wanderers'
        }

understat_teams = [team.replace(" ", "_") for team in list(team_names.values())]
len(understat_teams)

20

In [224]:
def get_understat_team_players(team_name):
    """Scrapes player data (ID, Name) for a given team from Understat."""
    url = f"https://understat.com/team/{team_name}/2024"
    # premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

    driver = webdriver.Firefox()
    driver.get(url)
    players_el = driver.find_elements(By.CSS_SELECTOR, ".player-title>a")

    player_dets = []

    for player in players_el:
        player_id = player.get_attribute("href").split("/")[-1]
        player_name = player.text

        player_dets.append({"understat_id": player_id, "understat_name": player_name, "understat_team": team_name})


    # print(f"Found {len(player_dets)} players for {team_name}", player_dets)

    driver.close()
    return player_dets


# Get all Understat players
understat_player_data = []
for team in understat_teams:
    team_data = get_understat_team_players(team)
    if team_data is not None:
        understat_player_data.extend(team_data)

# Combine all teams into one DataFrame
understat_df = pd.DataFrame(understat_player_data)
understat_df

,understat_id,understat_name,understat_team
0,5220,Kai Havertz,Arsenal
1,7752,Gabriel Martinelli,Arsenal
2,7322,Bukayo Saka,Arsenal
3,5304,Mikel Merino,Arsenal
4,7698,Leandro Trossard,Arsenal
...,...,...,...
536,10945,Santiago Bueno,Wolverhampton_Wanderers
537,13022,André,Wolverhampton_Wanderers
538,13092,Carlos Forbs,Wolverhampton_Wanderers
539,13117,Alfie Pond,Wolverhampton_Wanderers


In [225]:
# Get FPL data from the official API
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
data = response.json()

# Extract teams and players
teams = {team["id"]: team["name"] for team in data["teams"]}
players = data["elements"]

# Function to get players from a specific team
def get_players_for_team(team_name):
    team_id = next((id for id, name in teams.items() if name.lower() == team_name.lower()), None)
    # print(team_id)
    if team_id is None:
        return f"Team '{team_name}' not found."

    team_players = [player for player in players if player["team"] == team_id]

    return [
        {'fpl_id': player['id'] ,"fpl_name": f"{player['first_name']} {player['second_name']}", "fpl_position": player["element_type"], 'team': team_name}
        for player in team_players
    ]

# Example: Get players from Manchester City
# team_name = 'Man City'
players_list = [] # get_players_for_team(team_name)

for team in list(teams.values()):
    players_list.extend(get_players_for_team(team))


for player in players_list:
        player['full_team_name'] = team_names[player['team']]

fpl_data = pd.DataFrame(players_list)




In [226]:
import pandas as pd
# from fuzzywuzzy import process

# Sample DataFrames
# df1 = pd.DataFrame({'Player Name': ['Cristiano Ronaldo', 'Messi', 'Neymar', 'Mbappe']})
# df2 = pd.DataFrame({'FPL Name': ['C. Ronaldo', 'Lionel Messi', 'Neymar Jr', 'K. Mbappé'],
#                     'FPL ID': [1, 2, 3, 4]})

# Function to find the best match using fuzzy matching
def fuzzy_merge(df1, df2, key1, key2, threshold=80):

    # Find names that didn't match
    missing_players ={
        "Jorginho" : "Jorge Luiz Frello Filho",
        "Emiliano Martinez": "Emiliano Martínez Romero",
        "Morato": "Felipe Rodrigues da Silva",
        "Chimuanya Ugochukwu": "Lesley Ugochukwu",
        "Tomas Soucek": "Tomáš Souček",
        "Cheick Oumar Doucoure": "Cheick Doucouré",
        "Beto": "Norberto Bercique Gomes Betuncal",
        "Chiquinho": "Francisco Jorge Tomás Oliveira",
        "Casemiro": "Carlos Henrique Casimiro"
    }

    matches = []

    for name in df1[key1]:
        if name in missing_players:
            best_match = missing_players[name]
            matched_row = df2[df2[key2] == best_match].iloc[0].to_dict()
            matched_row[key1] = name  # Keep original name
            matched_row['understat_id'] = df1[df1[key1] == name].iloc[0]['understat_id']
            # matched_row['other_name'] = None
            matches.append(matched_row)
        else:
            match = process.extractOne(name, df2[key2])  # Find best match

            if match:  # Ensure match is not None
                best_match, score, other = match  # Unpack correctly

                if score >= threshold:  # Only keep matches above threshold
                    matched_row = df2[df2[key2] == best_match].iloc[0].to_dict()
                    matched_row[key1] = name  # Keep original name
                    matched_row['understat_id'] = df1[df1[key1] == name].iloc[0]['understat_id']
                    matches.append(matched_row)




    return pd.DataFrame(matches)

# Merge DataFrames using fuzzy matching
merged_df = fuzzy_merge(understat_df, fpl_data, "understat_name", "fpl_name")


In [227]:
merged_df

,fpl_id,fpl_name,fpl_position,team,full_team_name,understat_name,understat_id
0,4,Kai Havertz,4,Arsenal,Arsenal,Kai Havertz,5220
1,9,Gabriel Martinelli Silva,3,Arsenal,Arsenal,Gabriel Martinelli,7752
2,17,Bukayo Saka,3,Arsenal,Arsenal,Bukayo Saka,7322
3,633,Mikel Merino,3,Arsenal,Arsenal,Mikel Merino,5304
4,23,Leandro Trossard,3,Arsenal,Arsenal,Leandro Trossard,7698
...,...,...,...,...,...,...,...
536,564,Santiago Bueno,2,Wolves,Wolverhampton Wanderers,Santiago Bueno,10945
537,731,Andrés García,2,Aston Villa,Aston Villa,André,13022
538,35,Diego Carlos Santos Silva,2,Aston Villa,Aston Villa,Carlos Forbs,13092
539,664,Alfie Pond,2,Wolves,Wolverhampton Wanderers,Alfie Pond,13117


In [228]:
# Check for missing players and rerun the fuzzy matching
under_after = [i for i in merged_df['understat_id'].tolist()]
under_before = [i for i in understat_df['understat_id'].tolist()]

# find the difference between the two lists
diff = list(set(under_before) - set(under_after))
diff

understat_df[understat_df['understat_id'].isin(diff)]

,understat_id,understat_name,understat_team


In [229]:
merged_df.to_csv('fpl_understat_id_name.csv', index=False)